In [2]:
# Download and unpack data
from rdkit import Chem
import utils.data_prep.config as c
from utils.data_prep.download import download

download()
print(c.PROJECT_DIR)

/home/xduan7/PycharmProjects/MoReL


In [3]:
# Load all the CID and perform train/test/split
RAND_SEED = 0
TEST_SIZE = 5000

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

pcba_cid_array = pd.read_csv(
    c.PCBA_CID_FILE_PATH,
    sep='\t',
    header=0,
    index_col=None,
    usecols=[0]).values.reshape(-1).astype(np.int32)

trn_cid_array, val_cid_array = train_test_split(
    pcba_cid_array, 
    test_size=c.VALIDATION_SIZE, 
    random_state=c.RANDOM_STATE)

# Transforming all the array into set for faster lookup
# l = list(range(4000000))
# a = np.array(l)
# s = set(l)
# %timeit random.randint(0, 3999999) in l
# 19.4 ms ± 3.32 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
# %timeit random.randint(0, 3999999) in a
# 3.04 ms ± 55.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# %timeit random.randint(0, 3999999) in s
# 1.33 µs ± 7.42 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

pcba_cid_set = set(pcba_cid_array)
trn_pcba_cid_set = set(trn_cid_array)
val_pcba_cid_set = set(val_cid_array)


In [5]:
import os
import json
from rdkit import Chem

# Sadly there is no cPicle in python 3
import pickle

CHUNK_SIZE = 2 ** 16

# TODO: This is a very stupid way for trying to locate the chunk of a CID
# Note that the CID is strictly incremental, which we can take advantage of. 
CID_MOL_FILE_NAME = './CID-Mol/Mol/CID-mol_%04i.pkl'
CID_MOL_FILE_PATH = os.path.join(c.PROCESSED_DATA_DIR, CID_MOL_FILE_NAME)

# Need to specify the FP (probably using ECFP4(1024) + ECFP6(1024))
# CID_ECFP_FILE_NAME
# CID_ECFP_FILE_PATH

# Need to decide either using sparse matrix of dense one
# CID_GRAPH_FILE_NAME
# CID_GRAPH_FILE_NAME

CHUNK_CID_FILE_NAME = './CID-Mol/chunk_num-CID.txt'
CHUNK_CID_FILE_PATH = os.path.join(c.PROCESSED_DATA_DIR, CHUNK_CID_FILE_NAME)
UNUSED_CID_FILE_NAME = './CID-Mol/unused_CID.txt'
UNUSED_CID_FILE_PATH = os.path.join(c.PROCESSED_DATA_DIR, 
                                    UNUSED_CID_FILE_NAME)

# cid_mol = [] 
unused_cid = []
chunk_num_cid_dict = {}

for chunk_idx, cid_inchi_df_chunk in enumerate(
        pd.read_csv(c.CID_INCHI_FILE_PATH,
                    sep='\t',
                    header=None,
                    index_col=[0],
                    usecols=[0, 1],
                    chunksize=CHUNK_SIZE)):
    
    print("writing to file ... ")
    cid_inchi_df_chunk.to_hdf('./data/tmp.h5', 'tmp')
    
    cid_mol_chunk = [] 
    for cid, row in cid_inchi_df_chunk.iterrows():
        
        
        if cid not in pcba_cid_set:
            continue
        
        inchi = row[1]
        try:
            mol = Chem.MolFromInchi(inchi)
            assert mol
            
            # Extract the following features:
            #  * SMILES, 
            #  * fingerprint
            #  * graph (nodes and sparse adjacency matrix)
            
            # SMILES
            # (do we encode the SMILES here or compute on the fly?)
            # smiles = Chem.MolToSmiles()
            # assert len(smiles) != 0
            
            # Fingerprint 
            # # Encoding to hex bit strings
            # bit_str = AllChem.GetMorganFingerprintAsBitVect(
            #         mol, radius=radius, nBits=FP_N_BITS,
            #         useFeatures=useFeatures).ToBitString()
            # hex_str = hex(int(bitStr, 2))
            # 
            # # Decode to feature (list of 0, 1)
            # bit_str = str(bin(int(hex_str, 16)))[2:]
            # feature = np.concatenate(
            #     (np.zeros(shape=(1024 - len(bit_str))),
            #      np.array([np.int(c) for c in bit_str])), axis=0)
            
            # TODO
            # Graph 
            # (do we encode the graph here or compute on the fly?)
            
            # Will only save this molecule and its features if all the 
            # features are valid. 
            
        except AssertionError:
            # print('Failed converting compoud (CID=%i) to molecule.' % cid)
            unused_cid.append(cid)
            continue
        
        # if chunk_idx in chunk_num_cid_dict:
        #     chunk_num_cid_dict[chunk_idx].append(cid)
        # else:
        #     chunk_num_cid_dict[chunk_idx] = [cid, ]
        
        cid_mol_chunk.append(row)

    # with open(CID_MOL_FILE_PATH % chunk_idx, 'w+b') as f:
    #     pickle.dump(cid_mol_chunk, f, pickle.HIGHEST_PROTOCOL)

with open(CHUNK_CID_FILE_PATH, 'w') as f:
    json.dump(chunk_num_cid_dict, f, indent=4)



writing to file ... 


writing to file ... 


writing to file ... 


writing to file ... 


writing to file ... 


...

writing to file ... 


writing to file ... 


writing to file ... 


writing to file ... 


KeyboardInterrupt: 